# Correlating Oil and Gas Drilling with Seismicity

---

*Skip ahead to the map for 2015 [here](https://rawgit.com/aaronzira/earthquakes/master/map.html), which is to large to keep in this notebook.*

---

## Background and motivation

Having studied geology as an undergraduate, and having personally experienced seismic events of moment magnitudes [6.7](https://en.wikipedia.org/wiki/1994_Northridge_earthquake) and [9.0](https://en.wikipedia.org/wiki/2011_T%C5%8Dhoku_earthquake_and_tsunami), the prospect of working with earthquake data was very enticing for me. Articles such as [this one](http://www.scientificamerican.com/article/drilling-for-earthquakes/) from Scientific American highlight the dramatic increase in number of earthquakes in the Central US over the last few decades (as seen in the USGS image below), and their potential correlations with drilling operations in the area: 

>"Researchers at the USGS and other institutions have tied earthquake surges in eight states, including **Texas, Oklahoma, Ohio, Kansas and Arkansas,** to oil and gas operations..."

[<img src="http://earthquake.usgs.gov/research/induced/images/hockey-stick.gif" alt="Cumulative central US earthquakes 1973-2016" title="Cumulative central US earthquakes 1973-2016" style="width: 650px;"/>](http://earthquake.usgs.gov/research/induced/)

My goal with this project was twofold -- merge disparate data sources to:
1. create an easy-to-digest visualization for a casual observer to learn from
2. make that data available for future machine learning projects

The [USGS earthquake catalog](http://earthquake.usgs.gov/fdsnws/event/1/) seemed an excellent source of unbounded data that could be compared against static data on oil and gas wells to accomplish my goals. 

### Note
In order to ensure compatibility with [Apache Spark](http://spark.apache.org/), this project makes use of a Python 2 kernel (2.7.12).

## Imports

In [ ]:
from __future__ import print_function,unicode_literals
import json
import datetime
import time

import pandas as pd
import PyPDF2
import numpy as np
import requests
from scipy import spatial
from shapely.geometry import shape,Point
import folium

## Wells: create tables, find counties

With this great [index of wells data by state](http://pmc.ucsc.edu/~brodsky/wellindex.html), I was able to find which states had data available and where. Many states charge for the data or have broken links, but thankfully Colorado, Kansas, and Oklahoma had theirs readily available and are contiguous, so that seemed a perfect place to start.

### Download zip/shape files, read in as Pandas DataFrame objects

The .txt and .xlsx files of wells from Kansas and Oklahoma are no problem for [Pandas](http://pandas.pydata.org/)' `.read_csv()` and `.read_excel()` functions, but the GIS shapefiles for Colorado needed to be converted to something that Python could work with more easily.

[Colorado](http://cogcc.state.co.us/documents/data/downloads/gis/WELLS_SHP.ZIP) -> converted to .csv file with [mapshaper](http://www.mapshaper.org/)

[Oklahoma](http://pmc.ucsc.edu/~ivmiller/UIC.xlsx.zip)

[Kansas](http://www.kgs.ku.edu/PRS/Ora_Archive/ks_wells.zip "ks_wells.zip")

In [ ]:
co = pd.read_csv('./eq_project/wells/wells.csv')
ks = pd.read_csv('./eq_project/wells/ks_wells.txt')
ok = pd.read_excel('./eq_project/wells/UIC.xlsx')

```python
# Each state table has many features, many of which were not used for this phase of the project
print('Colorado:',co.columns.values)
print('Kansas:',ks.columns.values)
print('Oklahoma:',ok.columns.values)
```
```
Colorado: ['API' 'API_Label' 'Operator' 'Well_Title' 'Facil_Id' 'Facil_Type'
 'Facil_Stat' 'Operat_Num' 'Well_Num' 'Well_Name' 'Field_Code' 'Dist_N_S'
 'Dir_N_S' 'Dist_E_W' 'Dir_E_W' 'Qtr_Qtr' 'Section' 'Township' 'Range'
 'Meridian' 'Latitude' 'Longitude' 'Ground_Ele' 'Utm_X' 'Utm_Y' 'Loc_Qual'
 'Field_Name' 'Api_Seq' 'API_County' 'Loc_ID' 'Loc_Name' 'Spud_Date'
 'Citing_Typ' 'Max_MD' 'Max_TVD']
Kansas: ['KID' 'API_NUMBER' 'API_NUM_NODASH' 'LEASE' 'WELL' 'FIELD' 'LATITUDE'
 'LONGITUDE' 'LONG_LAT_SOURCE' 'TOWNSHIP' 'TWN_DIR' 'RANGE' 'RANGE_DIR'
 'SECTION' 'SPOT' 'FEET_NORTH' 'FEET_EAST' 'FOOT_REF' 'ORIG_OPERATOR'
 'CURR_OPERATOR' 'ELEVATION' 'ELEV_REF' 'DEPTH' 'PRODUCE_FORM' 'IP_OIL'
 'IP_GAS' 'IP_WATER' 'PERMIT' 'SPUD' 'COMPLETION' 'PLUGGING' 'MODIFIED'
 'OIL_KID' 'OIL_DOR_ID' 'GAS_KID' 'GAS_DOR_ID' 'KCC_DOCKET' 'STATUS'
 'STATUS2' 'COMMENTS']
Oklahoma: ['API_COUNTY' 'API_NUMBER' 'LEASE_NAME' 'WELL_NUMBER' 'DECIMAL_LAT'
 'DECIMAL_LONG' 'YEAR_1012A' 'PACKERDPTH' 'JAN' 'PRESSURE' 'VOLUME' 'FEB'
 'PRESSURE_1' 'VOLUME_1' 'MARCH' 'PRESSURE_2' 'VOLUME_2' 'APRIL'
 'PRESSURE_3' 'VOLUME_3' 'MAY' 'PRESSURE_4' 'VOLUME_4' 'JUNE' 'PRESSURE_5'
 'VOLUME_5' 'JULY' 'PRESSURE_6' 'VOLUME_6' 'AUGUST' 'PRESSURE_7' 'VOLUME_7'
 'SEPT' 'PRESSURE_8' 'VOLUME_8' 'OCT' 'PRESSURE_9' 'VOLUME_9' 'NOV'
 'PRESSURE_10' 'VOLUME_10' 'DEC' 'PRESSURE_11' 'VOLUME_11' 'Unnamed: 44'
 'Unnamed: 45' 'Unnamed: 46' 'Unnamed: 47' 'Unnamed: 48' 1977 3141810]
```

### Deciding on relevant features

I had originally intended to make use of features such as depth, time since spudding, and the name on the lease, but ultimately determined that because I had less than two weeks for the project, [API number](https://en.wikipedia.org/wiki/API_well_number), location (latitude and longitude), and county were the only important features for this project. While Colorado and Oklahoma have columns of API codes that indicate the county in which the surface of a well is found, for Kansas there is only information on Township, which was not relevant to me. I ended up cross checking the latitude and longitude of each well in that state against a [GeoJSON file](http://geojson.org/) that essentially splits the state into one polygon for each county.

In [ ]:
# Keep important features only
co = co[['API','API_County','Latitude','Longitude']]
ks = ks[['API_NUMBER','LATITUDE','LONGITUDE']]
ok = ok[['API_NUMBER','API_COUNTY','DECIMAL_LAT','DECIMAL_LONG']]

```python
print('Colorado:\n\t',co.columns.values,'\n\t\tx',co.shape[0],'wells')
print('Kansas:\n\t',ks.columns.values,'\n\t\tx',ks.shape[0],'wells')
print('Oklahoma:\n\t',ok.columns.values,'\n\t\tx',ok.shape[0],'wells')

```
```
Colorado:
	 ['API' 'API_County' 'Latitude' 'Longitude'] 
		x 110051 wells
Kansas:
	 ['API_NUMBER' 'LATITUDE' 'LONGITUDE'] 
		x 474515 wells
Oklahoma:
	 ['API_NUMBER' 'API_COUNTY' 'DECIMAL_LAT' 'DECIMAL_LONG'] 
		x 144941 wells
```

### Determine county of each well

The well databases of the three states I retrieved did not include the county of a given well, and I needed that in order to generate a [choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) of the three states with counties shaded based on the number of wells within them. The Oklahoma and Colorado databases contained columns with the API county code for each well, which I converted to county name using [this api codes pdf](https://dl.ppdm.org/dl/796) and the [PyPDF2 module](https://pythonhosted.org/PyPDF2/) to extract the text.

In [ ]:
pdf = PyPDF2.PdfFileReader(open('./eq_project/codes.pdf',"rb"))

In [ ]:
def api_code_extractor(pages,state,first_line,last_line):
    
    """Extracts the codes and corresponding county names from the API codes PDF 
    for a given state, and returns a data frame of the results.  The relevant 
    pages and line numbers must be determined manually by examining the lines 
    retrieved by the PyPDF2 module.  The formatting of county names is designed 
    to match the keys in the state GeoJSON files from CivicDashboards at 
    http://catalog.opendata.city/dataset/
        
    Args:
        pages (list): Ordered page numbers for the current state.
        state (str): The two-letter state code of the current state.
        first_line (int): The line number of the first API code for the 
            current state (must be determined manually).
        last_line (int): The line number of the last county name for the 
            current state (must be determined manually).
    
    Returns:
        pandas DataFrame: A table for the current state with columns of codes 
            and county names.
    """
    
    
    dic = {}
    pg_count = len(pages)
    
    for loop,page_num in enumerate(pages):
        page = pdf.getPage(page_num)        
        
        # These logical checks are overkill for the states I used, but are 
        # put in place in hopes that data for other states would be added 
        # to the analysis. 
        
        # Assume last page by default
        first = 10
        last = last_line

        # Any page other than last
        if pg_count-(loop+1) > 0:
            last = -1
            
            # First page only
            if not dic: 
                # If state has less than one full page of counties
                if pg_count == 1:
                    last = last_line
                first = first_line
        
        # Zip together codes and matching counties, update dictionary with 
        # codes as keys and properly formatted county names as values.
        for code,county in zip(page.extractText().splitlines()[first:last:4],
                               page.extractText().splitlines()[first+1:last:4]):
            dic[int(code)] = ' '.join([word[0]+word[1:].lower() for word in 
                                county.split()])+' County, {}'.format(state)
    
    df = pd.DataFrame.from_dict({'Code':dic}).reset_index()
    df.columns = ['Code','County']
    
    return df

After retrieving the codes and corresponding counties and storing them as data frames, I joined them with the Colorado and Oklahoma tables such that the county is included with the other relevant data for each well.

In [ ]:
co_codes = api_code_extractor([5,6],'CO',first_line=38,last_line=86)
ok_codes = api_code_extractor([44,45],'OK',first_line=94,last_line=192)

In [ ]:
ok['COUNTY'] = ok.merge(ok_codes,how='left',
                        left_on='API_COUNTY',right_on='Code')['County']
co['County'] = co.merge(co_codes,how='left',
                        left_on='API_County',right_on='Code')['County']
ok.dropna(inplace=True)
co.dropna(inplace=True)
ks.dropna(inplace=True)

Aside from matching the formatting 

>'[COUNTY NAME] County, [STATE ABBREVIATION]'

in the GeoJSON files linked to in the section below on creating the map, there were two small inconsistencies between the PDF and the key names, so I manually replaced those.

In [ ]:
ok['COUNTY'] = ok['COUNTY'].replace({
                        'Mc Clain County, OK':'McClain County, OK',
                        'Mc Intosh County, OK':'McIntosh County, OK'})

Determining counties for Kansas wells was not as straightforward. Thanks to explanations from users 'Zebs' and 'wf.' on [this stack overflow thread](http://stackoverflow.com/questions/20776205/point-in-polygon-with-geojson-in-python), I was able to use the [Shapely](https://pypi.python.org/pypi/Shapely) module and read in the boundaries of Kansas' county polygons from [this GeoJSON file](http://catalog.opendata.city/fa_IR/dataset/kansas-counties-polygon/resource/815cebfa-5666-4f7d-943c-f53945a4347e "Kansas GeoJSON file"), and loop through each point (latitude and longitude of a well) to determine which polygon (and therefore county) contained it.

In [ ]:
# Create dictionary of Kansas counties with county name as key, 
# corresponding boundary outline (as Shapely shape object) as value.
polygons = {}

with open('./eq_project/counties/kansas.geojson','r') as f:
    js = json.load(f)

for feature in js['features']:
    polygons[feature['properties']['name']] = shape(feature['geometry'])

In [ ]:
# Dictionary of Shapely point objects of each well based on its lat and long, 
# using the well's API number as the key, and the long/lat coordinates as the 
# value (lat and long are flipped with Shapely).
pts = {}
for point in ks.values:
    pts[point[0]]=(Point(point[[2,1]]))

While I'd rather have not used nested ```for``` loops below, for my purposes 3-4 minutes was 
not a big price to pay, and the logic is simple to follow: looking at one 
well at a time, loop through all counties until finding the first one that 
contains it, and assign to a new dictionary a key of that well's API number, 
and a value of the county in which it was found.  The ```break``` in the loop 
saves time, as only one county will contain any one well, so there is no 
reason to continue searching after finding a match. Given more time, I would 
have chosen a data structure that would speed this up.

In [ ]:
ks_well_counties = {}
for api,coordinates in pts.items():
    for county,polygon in polygons.items():
        if polygon.contains(coordinates):
            ks_well_counties[api] = county
            break

In [ ]:
# Kansas API and county database
ks_with_counties = pd.DataFrame(
                ([api,county] for api,county in ks_well_counties.items()),
                columns=['API','County'])

In creating the final map I could not ignore missing keys in the GeoJSON file, so I had to manually set values for those counties to 0; i.e., there are no wells in the following counties:

In [ ]:
no_wells = pd.DataFrame({'County': np.array([
                        'Johnston County, OK','Ottawa County, OK',
                        'Pushmataha County, OK','Adair County, OK',
                        'Cherokee County, OK','Delaware County, OK',
                        'Choctaw County, OK','McCurtain County, OK',
                        'Teller County, CO','Rio Grande County, CO',
                        'Summit County, CO','Mineral County, CO',
                        'Clear Creek County, CO','Lake County, CO',
                        'San Juan County, CO','Hinsdale County, CO',
                        'Gilpin County, CO']),
                        'Number': np.array([0]*17)})

### Write out cleaned tables

At this point I had the wells tables cleaned up as I wanted them, so I wrote them out to CSV (as .txt) files to have as backups and for working with [Spark](http://spark.apache.org/).

In [ ]:
ok.to_csv('./eq_project/CSVs/oklahoma.txt',header=False,index=False)
co.to_csv('./eq_project/CSVs/colorado.txt',header=False,index=False)
ks.to_csv('./eq_project/CSVs/kansas.txt',header=False,index=False)
ks_with_counties.to_csv('./eq_project/CSVs/kansas_counties.txt',header=False,index=False)
no_wells.to_csv('./eq_project/CSVs/no_wells.txt',header=False,index=False)

## Using Spark

Though I only accessed the data locally, Apache Spark allows one to take advantage of the very powerful, fault-tolerant [RDD](https://spark.apache.org/docs/0.8.1/api/core/org/apache/spark/rdd/RDD.html "Resilient Distributed Dataset") and associated capabilities, crucially to perform parallel operations across a cluster of machines on massive amounts of data, without needing to think much about optimization. In this case, it was for practice. In order to start a Spark-enabled IPython notebook from the terminal, one can run:

```
IPYTHON_OPTS="notebook" pyspark```

```python
# Test connection to Spark context:
sc
```
```
<pyspark.context.SparkContext at 0x1090a3610>
```

In [ ]:
# Read in CSV files as RDDs
oklahoma = sc.textFile('./eq_project/CSVs/oklahoma.txt')
colorado = sc.textFile('./eq_project/CSVs/colorado.txt')
kansas = sc.textFile('./eq_project/CSVs/kansas.txt')
kansas_counties = sc.textFile('./eq_project/CSVs/kansas_counties.txt')
no_wells_rdd = sc.textFile('./eq_project/CSVs/no_wells.txt')

To get RDDs of just the data I needed at this point, I dropped the API county code columns from the Colorado and Oklahoma tables, and extracted properly typed data from each line. The `API` field for Colorado included the county code as a prefix, so I only took the last five digits. For Kansas, the `API_NUMBER` field included several extra codes, the longest of which was the API number alone, so I only kept that. In addtion, it was at this point that I joined the two Kansas tables (one with API and location, the other with API and county).

In [ ]:
# Adjust formatting to take advantage of Spark's key-value pair operations
kansas = kansas.map(
    lambda line:line.split(',')).map(
    lambda _list:(str(_list[0]),float(_list[1]),float(_list[2]))).map(
    lambda row:(row[0],[row[1],row[2]]))

kansas_counties = kansas_counties.map(
    lambda line:line.split(',')).map(
    lambda _list:(str(_list[0]),str(_list[1][1:]+','+_list[2][:-1])))

In [ ]:
kansas = kansas.join(kansas_counties).map(
    lambda _list:[int(_list[0].split('-')[
                    np.argmax(map(len,_list[0].split('-')))]),
                  float(_list[1][0][0]),float(_list[1][0][1]),
                  str(_list[1][1])])

colorado = colorado.map(
    lambda line:line.split(',')).map(
    lambda _list:[int(_list[0][-5:]),
                  float(_list[2]),float(_list[3]),
                  str(','.join((_list[4],_list[5])))[1:-1]])

oklahoma = oklahoma.map(
    lambda line:line.split(',')).map(
    lambda _list:[int(_list[0]),
                  float(_list[2]),float(_list[3]),
                  str(','.join((_list[4],_list[5])))[1:-1]])

no_wells_rdd = no_wells_rdd.map(
    lambda line:(str(line[1:-3]),int(line[-1])))

```python
# Check formatting
print(oklahoma.first())
print(kansas.first())
print(colorado.first())
```
```
[21680, 34.6907726, -97.057519, 'Garvin County, OK']
[19926, 37.91039, -95.09883, 'Allen County, KS']
[8888, 40.419416, -102.379999, 'Yuma County, CO']
```

I then combined the three states RDDs into one, and determined the number of times each county name appears (i.e., how many wells each county has) to use in shading the choropleth map. Here I also needed to include the counties in which there were no wells.

In [ ]:
# All three states combined
wells_db = sc.union([oklahoma,colorado,kansas])

In [ ]:
# Number of wells per county
all_wells_per_county = wells_db.map(
    lambda row:(row[3],1)).reduceByKey(
    lambda x,y:x+y).union(no_wells_rdd)

```python
# Confirm number of counties
all_wells_per_county.count()
```
```
247
```

# Find earthquakes near wells

Now that the data on all of the wells was in a cleaned, convenient format, the next step was to find a data structure that could hold this information and allow for calculation of distances between earthquakes and the wells. After a number of dead ends, I settled on the [K-D tree](https://en.wikipedia.org/wiki/K-d_tree), which would allow me to see each well as a point in latitude-longitude space, so comparing against coordinates of an earthquake's epicenter would be very simple. For this project I used the `KDTree` object from [scipy.spatial](http://docs.scipy.org/doc/scipy/reference/spatial.html).

## Create directory and K-D tree of all wells in the three states

In [ ]:
wells_map = {}
for api,lat,lng,county in wells_db.collect():
    wells_map[lat,lng] = api
    
locations = (wells_map.keys())
wells_tree = spatial.KDTree(locations)

## Find nearest well from an earthquake

This function is used within another (`eq_to_map()`) that is defined below, in returning pertinent information on the nearest well to an earthquake.

In [ ]:
def get_nearest_well(point,tree,directory):
    """Returns information on the nearest well to a given earthquake.
    
    Args:
        point (list): Latitude and longitude of an earthquake.
        tree (Scipy spatial K-D tree): Well locations to search.
        directory (dict): Lat/long coordinates of wells in the K-D tree and 
            corresponding API numbers.
    
    Returns:
        str: Distance in km from the earthquake to the nearest well.  The 
            distance is calculated by converting one decimal degree to an 
            approximate 111.32km.
    """
    
    distance,index = tree.query(point)
    return str(distance*111.32) +' km '+ 'from Well # '+ \
        str(directory[tree.data[index][0],tree.data[index][1]])

## Read in earthquakes data from USGS

This function retrieves recent earthquake data from the USGS, and renders it usable by `eq_to_map()` to plot on the choropleth maps created below.

In [ ]:
def quake_to_rdd(start = datetime.datetime.fromtimestamp(
                    time.time()-86400).strftime('%Y-%m-%dT%H:%M:%S'),
                stop = datetime.datetime.fromtimestamp(
                    time.time()).strftime('%Y-%m-%dT%H:%M:%S'),
                magnitude = 1.5, lats = [33,41.5], longs = [-109.5,-94]):    
    """Creates an RDD of worldwide earthquakes within a given time range, above 
    a given magnitude, and bounded by given latitudes and longitudes.
    
    Args:
        start (str): YYYY-MM-DD (with optional HH:MM:SS, joined by 'T' without 
            quotes) of first date to search.  Defaults to one day 
            (86400 seconds) before present.
        stop (str): YYYY-MM-DD (with optional HH:MM:SS, joined by 'T' without 
            quotes) of last date to search.  Defaults to present.
        magnitude (float): Minimum magnitude to search.  Defaults to 1.5.
        lats (list): Minimum and maximum latitudes to search. 
            Default to 33 and 41.5.
        longs (list): Minimum and maximum longitudes to search. 
            Default to -109.5 and -94.
    
    Returns:
        Pyspark RDD: Most important info on each earthquake returned from the 
            search, in JSON format.
    """
    
    url = ('http://earthquake.usgs.gov/fdsnws/event/1'
            '/query?format=geojson&starttime={0}&endtime={1}'
            '&minmagnitude={2}'
            '&minlatitude={3}&maxlatitude={4}'
            '&minlongitude={5}&maxlongitude={6}'
          ).format(start,stop,magnitude,lats[0],lats[1],longs[0],longs[1])
    results = []
    for event in requests.get(url).json()['features']:
        earthquake = {}
        # Earthquakes are returned with longitude first, latitude second.
        earthquake['long'], earthquake['lat'], earthquake['depth'] = \
            event['geometry']['coordinates'][0], \
            event['geometry']['coordinates'][1], \
            event['geometry']['coordinates'][2]
        earthquake['title'] = event['properties']['title']
        earthquake['magnitude'] = event['properties']['mag']
        # Earthquakes are returned from USGS in terms of milliseconds since 
        # 1 January 1970, requiring division by 1000 to match formatting in 
        # Python's datetime module.
        earthquake['timestamp'] = \
            datetime.datetime.fromtimestamp(
                event['properties']['time']/1000).strftime('%c')
        
        results.append(earthquake)
        
    return sc.parallelize(results)

```python
# Test with earthquakes from the past 24 hours, examine first three
last_day = quake_to_rdd()
last_day.take(3)
```
```
[{u'depth': 5,
  u'lat': 35.6625,
  u'long': -97.1556,
  u'magnitude': 3.5,
  u'timestamp': 'Mon Aug 15 23:08:14 2016',
  u'title': u'M 3.5 - 3km E of Luther, Oklahoma'},
 {u'depth': 5,
  u'lat': 36.4566,
  u'long': -98.7433,
  u'magnitude': 2.6,
  u'timestamp': 'Mon Aug 15 17:07:19 2016',
  u'title': u'M 2.6 - 31km NW of Fairview, Oklahoma'},
 {u'depth': 3.94,
  u'lat': 37.242,
  u'long': -97.6586667,
  u'magnitude': 1.81,
  u'timestamp': 'Mon Aug 15 07:12:10 2016',
  u'title': u'M 1.8 - 16km S of Conway Springs, Kansas'}]
```

# Create choropleth map

After exploring different options of map types and reading a few great tutorials such as [this](https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium/) and [this](https://ocefpaf.github.io/python4oceanographers/blog/2015/08/24/choropleth/), I decided that [Folium's](https://github.com/python-visualization/folium) choropleth map was best suited to my needs, as I wanted county-level aggregate numbers on wells, overlaid by interactive map markers for nearby earthquakes.

## State GeoJSON files

The Kansas file is the same as used above to determine the county of each well. All three states' files contain coordinates that bound each county within them. For this choropleth map, each county is shaded in proportion to the number of wells within its borders.

[Colorado](http://catalog.opendata.city/dataset/colorado-counties-polygon/resource/c9ddc844-6d01-4c7c-8c98-df932ea94597) -> 'eq_project/counties/colorado.geojson'

[Kansas](http://catalog.opendata.city/fa_IR/dataset/kansas-counties-polygon/resource/815cebfa-5666-4f7d-943c-f53945a4347e) -> 'eq_project/counties/kansas.geojson'

[Oklahoma](http://catalog.opendata.city/dataset/oklahoma-counties-polygon/resource/75b87ccf-da9e-464e-814b-16985041d2ca) -> 'eq_project/counties/oklahoma.geojson'

As I wanted one map that showed all three states together, I merged the individual GeoJSON files with [this script](https://gist.github.com/themiurgo/8687883 "merger.py"), run from the terminal as follows:

```shell
python merger.py -o merged.geojson colorado.geojson oklahoma.geojson kansas.geojson
```

## Choropleth map instance with earthquake data
The map is centered south of Manter, KS, which is roughly at the center of the three states, and the coloring/shading scale ranges from yellow for zero wells, to green, to blue. I lowered the upper threshold to 14000 wells, as there is only one county with more than 25000 (Weld County, CO - 35649; shown below), and I wanted to group the top ten or so counties together, which is more telling than having only one county colored blue on the map.

```python
all_wells_per_county.filter(
    lambda (county,count):count>14000).map(
    lambda (county,count):[count,county]).sortByKey(
    ascending=False).collect()
```
```
[(35649, 'Weld County, CO'),
 (24538, 'Carter County, OK'),
 (21747, 'Montgomery County, KS'),
 (19751, 'Allen County, KS'),
 (17037, 'Barton County, KS'),
 (16689, 'Stephens County, OK'),
 (16072, 'Miami County, KS'),
 (15898, 'Garfield County, CO'),
 (15421, 'Ellis County, KS'),
 (14999, 'Neosho County, KS'),
 (14894, 'Butler County, KS'),
 (14643, 'Woodson County, KS')]
```

In [ ]:
def eq_to_map(data,tree,directory,center=[37.4,-102]):
    """Creates interactive markers of nearby earthquakes on a choropleth map, 
    then saves and returns the map.  'Nearby' means having an epicenter 
    within 10km of a well in the K-D tree.  For a qualifying earthquake, the 
    radius of the marker is proportional to the magnitude, and clicking on the 
    marker will display relevant information about it, as retrieved by 
    get_nearest_well(). 
    
    Args:
        data (Pyspark RDD): Most important information on earthquakes as 
            returned by quake_to_rdd().
        tree (Scipy spatial K-D tree): Well locations to search.
        directory (dict): Lat/long coordinates of wells in the K-D tree and 
            corresponding API numbers.
        center (list): Lat/long coordinates of the center of the map.  Defaults 
            to [36.7295,-102.5132].
    
    Returns: Folium choropleth map: States colored based on number of wells 
        within their counties, and earthquake markers plotted on top.
    
    Note: The map is reinstantiated with each call of the function, as Folium 
        does not provide a simple way to reset all custom markers or regenerate 
        the base map.
    """
    
    choro_map = folium.Map(location=center,zoom_start=6)
    choro_map.choropleth(
                        geo_str = json.dumps(json.load(open(
                            './eq_project/counties/merged.geojson','r'))),
                        data = all_wells_per_county.collect(),
                        columns = ['County','Number'],
                        key_on = 'properties.name',
                        fill_color = 'YlGnBu',
                        fill_opacity = 0.6,
                        line_opacity = 0.2,
                        threshold_scale = 
                            np.linspace(1,14000,6,dtype=int).tolist(),
                        legend_name = 'Wells per county')

    for quake in data.collect():
        # 10km * (1 degree/111.13km) = .08998 degrees
        if tree.query([quake['lat'],quake['long']])[0] < .08998:
            title = quake['title']
            time = quake['timestamp']
            folium.CircleMarker([quake['lat'],quake['long']],
                                popup=title+'\n--\n'+time+'\n--\n'\
                                        +get_nearest_well([quake['lat'],
                                                           quake['long']],
                                                           tree,directory),
                                radius=quake['magnitude']*2500,
                                color='#000000',
                                fill_color='#ff0000'
            ).add_to(choro_map)
    choro_map.save('./map.html')
    return choro_map

# Putting it all together

## Note
The .html map files are too large to upload here, but my code follows. The 2015 map can be found [here](https://rawgit.com/aaronzira/earthquakes/master/map.html).

In [ ]:
# This will always produce a map of the last 30 days' worth of earthquakes
# 2592000 seconds in 30 days
thirty_days_ago = datetime.datetime.fromtimestamp(
                            time.time()-2592000).strftime('%Y-%m-%dT%H:%M:%S')
last_month = quake_to_rdd(start=thirty_days_ago)
eq_to_map(last_month,wells_tree,wells_map)

I thought that one interesting application of these visualizations would be to compare one year's worth of earthquakes from the eighties, nineties, 2000s, and 2010s, spaced by 10 years. 

Note that because of the comparatively large number of earthquakes in 2015, creating that map below takes much longer than the others (several minutes).

In [ ]:
eighties = quake_to_rdd(start='1985-01-01',stop='1985-12-31')
eq_to_map(eighties,wells_tree,wells_map)

In [ ]:
nineties = quake_to_rdd(start='1995-01-01',stop='1995-12-31')
eq_to_map(nineties,wells_tree,wells_map)

In [ ]:
aughts = quake_to_rdd(start='2005-01-01',stop='2005-12-31')
eq_to_map(aughts,wells_tree,wells_map)

In [ ]:
tens = quake_to_rdd(start='2015-01-01',stop='2015-12-31')
eq_to_map(tens,wells_tree,wells_map)

# What next?

I would like to port all the data to [Postgres](https://www.postgresql.org/), and take advantage of the performance, speed, and data types and structures available there, such as K-D trees and lat/long points. In addition, my distance calculations could be made more accurate using the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula), which determines great circle distances by taking into account the radius of the earth, instead of considering the area in question a 2-dimensional map. Getting more data is pretty much always a good idea, and it would be nice to have data on wells from many other states, especially all of those for which the USGS has established correlations between drilling and seismicity. 

Using the data created here, it would be interesting to apply machine learning techniques to determine if there are certain features (depth, well type, years active or inactive, direction, presence of other wells nearby, etc.) that contribute to a higher likelihood of induced seismicity in a particular area. In addition, data from a geologic map could be included to examine what types of rock are found in and around the areas of interest, potentially allowing for predictive applcations of future seismic activity related to wells anywhere. 